### Distraction From Breath

In [2]:
import os

import numpy as np
import pandas as pd
import scipy.io as sio

In [ ]:
# set root directory
root_dir = "/Path/To/EMBODY_Study/Files" 

In [3]:
# set analysis name
analysis = "phase1_demo"

In [4]:
# set subjects
subjects = [124]

In [5]:
def condition_num_to_name(condition):
    
    """Converts numeric condition (input, int) to nominal condition (output, str)."""
    
    if condition == 1:
        condition_name = "breath"
    elif condition == 2:
        condition_name = "feet"
    elif condition == 3:
        condition_name = "stop"
    elif condition == 4:
        condition_name = "self"
    elif condition == 5:
        condition_name = "sounds"
    elif condition == 0:
        condition_name = "noise"

    return condition_name

--------

### Length of time to return to breath
#### mean & sd

In [6]:
def return_to_breath_transition():

    breath_has_occured = False
    distraction_duration = 0
    distraction_duration_list = []
    
    for index in range(1, len(meditation)):
        current_value = meditation[index]
        previous_value = meditation[index - 1]
                
        if current_value == 1:
            breath_has_occured = True
        
        if (current_value != 1) and (breath_has_occured):
            
            distraction_duration += 1       
            
        elif (current_value == 1):
            
            if distraction_duration > 0:
                distraction_duration_list.append(distraction_duration)
            
            distraction_duration = 0
            
    if distraction_duration > 0:
        distraction_duration_list.append(distraction_duration)
            
    return distraction_duration_list


-------

### Event Switches 
#### (3x3 matrix: B-B, B-MW, B-S, MW-MW, MW-B, S-B, S-MW, S-S)


In [7]:
# transitions from other states back to the breath (return to the breath)
# counting the transition mental states

def condition_transitions():
    # count transitions from any condition to any conditin
    
    columns = ["breath", "stop", "self"]
    index = ["breath", "stop", "self"]
    transition_chart = pd.DataFrame(0, columns=columns, index=index)
    
    last_full_condition = None
    
    for index in range(1, len(meditation) - 2):
                
        prev_1 = meditation[index - 1]
        current_value = meditation[index]
        next_1 = meditation[index + 1]
        next_2 = meditation[index + 2]
        
        # handling first three values -- if first three values are the same, set last_full_condition
        if index == 1 and (prev_1 == current_value) \
        and (current_value == next_1) and (current_value != 0):
            last_full_condition = meditation[index]

        # while there is no "last full condition" continue iterating through data points until we hit 3 in a row
        while last_full_condition == None:

            index += 1
            if (meditation[index] != 0) and \
               (meditation[index] == meditation[index + 1]) and \
               (meditation[index + 1] == meditation[index + 2]):
                last_full_condition = meditation[index]
        
        # check to see if the current value is different from the value before it (indicating a transition)
        # then check to see if the current_value is a true "event" (it persists at least 3 TRs)
        if (current_value != 0) and (prev_1 != current_value) and \
            (current_value == next_1) and (next_1 == next_2):
                  
            # assign the numeric conditions to their nominal values
            # for indexing the transition_chart
            current_condition = condition_num_to_name(current_value)
            last_condition = condition_num_to_name(last_full_condition)         
            transition_chart[last_condition][current_condition] += 1
            
            last_full_condition = current_value
    
    # transpose values so column is previous condition and row is current transition
    transition_chart = transition_chart.transpose()
    
    return transition_chart

--------

### Count Event Switches

In [8]:
# define all possible condition switches for output
output = pd.DataFrame(columns=["distract_breath_mean", "distract_breath_sd", "eventswitch_br_st", 
                      "eventswitch_br_sf", "eventswitch_st_br", "eventswitch_st_sf", 
                      "eventswitch_sf_br", "eventswitch_sf_st", "eventswitch_br_br", "eventswitch_st_st",
                      "eventswitch_sf_sf", "eventswitch_total"])


In [9]:
for i in range(len(subjects)):
    
    print("")
    print("[Subject %d]" % subjects[i])
    print("")
        
    # Define meditation data
    data = pd.read_csv("%s/%s/results/step2/%d/step2_event_clean.csv" % 
                       (root_dir, analysis, subjects[i]), header=None)
    
    meditation = list(data[1])
    
    # mean duration of distraction from breath
    breath_distraction_durations = np.array(return_to_breath_transition())
    print("Mean Duration of Distraction From Breath: %f seconds" % (np.mean(breath_distraction_durations)))
    output.loc[subjects[i], "distract_breath_mean"] = np.mean(breath_distraction_durations)
    print("Standard Deviation: %f seconds" % np.std(breath_distraction_durations, ddof=1))
    output.loc[subjects[i], "distract_breath_sd"] = np.std(breath_distraction_durations, ddof=1)
        
    
    # calculate and show event switches
    event_switches_df = condition_transitions()
    print("")
    print(event_switches_df)
    print("")
    
    # add event switches to output df
    output.loc[subjects[i], "eventswitch_br_st"] = event_switches_df.loc["breath", "stop"]
    output.loc[subjects[i], "eventswitch_br_sf"] = event_switches_df.loc["breath", "self"]
    output.loc[subjects[i], "eventswitch_st_br"] = event_switches_df.loc["stop", "breath"]
    output.loc[subjects[i], "eventswitch_st_sf"] = event_switches_df.loc["stop", "self"]
    output.loc[subjects[i], "eventswitch_sf_br"] = event_switches_df.loc["self", "breath"]
    output.loc[subjects[i], "eventswitch_sf_st"] = event_switches_df.loc["self", "stop"]
    output.loc[subjects[i], "eventswitch_br_br"] = event_switches_df.loc["breath", "breath"]
    output.loc[subjects[i], "eventswitch_st_st"] = event_switches_df.loc["stop", "stop"]
    output.loc[subjects[i], "eventswitch_sf_sf"] = event_switches_df.loc["self", "self"]
    
    
    # total number of event switches
    total_event_switches = int(event_switches_df["breath"].sum() +
                          event_switches_df["stop"].sum() +
                          event_switches_df["self"].sum())
    
    # add total event switches to output df
    output.loc[subjects[i], "eventswitch_total"] = total_event_switches

    print("Total Number of Event Switches: %d" % total_event_switches)
    print("")
    print("")
    print("------------------------------------------------------------")
    print("")


[Subject 124]

Mean Duration of Distraction From Breath: 16.600000 seconds
Standard Deviation: 17.401970 seconds

        breath  stop  self
breath       0     6     9
stop         8     0     4
self         7     6     1

Total Number of Event Switches: 41


------------------------------------------------------------



#### Rows: current event
#### Columns: counting next event

In [13]:
# write output to file named step3_distraction_fluctuation.csv
output.to_csv("%s/%s/results/step3_compile/step3_2_distraction_fluctuation.csv" % (root_dir, analysis))